In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
import pandas as pd
spark = SparkSession.builder.appName("KcHouseData").master("local[*]").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/12 16:02:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark import SparkFiles
 
url = "https://dysk.agh.edu.pl/s/nWNWeirnCJDQycp/download/kc_house_data.csv.gz"
spark.sparkContext.addFile(url)
 
df = spark.read.csv(SparkFiles.get("kc_house_data.csv.gz"), header=True, inferSchema=True)
df.show(5)


+----------+---------------+--------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|        id|           date|   price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|grade|sqft_above|sqft_basement|yr_built|yr_renovated|zipcode|    lat|    long|sqft_living15|sqft_lot15|
+----------+---------------+--------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|7129300520|20141013T000000|221900.0|       3|      1.0|       1180|    5650|   1.0|         0|   0|        3|    7|      1180|            0|    1955|           0|  98178|47.5112|-122.257|         1340|      5650|
|6414100192|20141209T000000|538000.0|       3|     2.25|       2570|    7242|   2.0|         0|   0|        3|    7|      2170|          400|   

In [3]:
pdf = df.toPandas()
print(pdf.info())
pdf.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             21613 non-null  int64  
 1   date           21613 non-null  object 
 2   price          21613 non-null  float64
 3   bedrooms       21613 non-null  int32  
 4   bathrooms      21613 non-null  float64
 5   sqft_living    21613 non-null  int32  
 6   sqft_lot       21613 non-null  int32  
 7   floors         21613 non-null  float64
 8   waterfront     21613 non-null  int32  
 9   view           21613 non-null  int32  
 10  condition      21613 non-null  int32  
 11  grade          21613 non-null  int32  
 12  sqft_above     21613 non-null  int32  
 13  sqft_basement  21613 non-null  int32  
 14  yr_built       21613 non-null  int32  
 15  yr_renovated   21613 non-null  int32  
 16  zipcode        21613 non-null  int32  
 17  lat            21613 non-null  float64
 18  long  

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [4]:
df2 = df.withColumn("date", to_timestamp(col("date"), "yyyyMMdd'T'HHmmss"))


In [5]:
df2.show()

+----------+-------------------+---------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|        id|               date|    price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|condition|grade|sqft_above|sqft_basement|yr_built|yr_renovated|zipcode|    lat|    long|sqft_living15|sqft_lot15|
+----------+-------------------+---------+--------+---------+-----------+--------+------+----------+----+---------+-----+----------+-------------+--------+------------+-------+-------+--------+-------------+----------+
|7129300520|2014-10-13 00:00:00| 221900.0|       3|      1.0|       1180|    5650|   1.0|         0|   0|        3|    7|      1180|            0|    1955|           0|  98178|47.5112|-122.257|         1340|      5650|
|6414100192|2014-12-09 00:00:00| 538000.0|       3|     2.25|       2570|    7242|   2.0|         0|   0|        3|    7|   

In [6]:
df2 = df2.withColumn("unixdate", unix_timestamp("date"))
df2 = df2.drop("id")
df2 = df2.drop("date")

In [7]:
pdf = df2.toPandas()
pdf.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,unixdate
0,221900.0,3,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,1413158400
1,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,1418083200
2,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,1424822400
3,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,1418083200
4,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,1424217600


In [8]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler

In [33]:
input_cols = df2.columns
input_cols.remove("price")


assembler = VectorAssembler(
    inputCols=input_cols,
    outputCol="features"
)

In [34]:

df_transformed = assembler.transform(df2)


In [35]:

lr = LinearRegression()\
  .setMaxIter(100)\
  .setRegParam(3.0)\
  .setElasticNetParam(0.5)\
  .setFeaturesCol("features")\
  .setLabelCol("price")
model = lr.fit(df_transformed)

In [36]:
print(f'RMSE: {model.summary.rootMeanSquaredError}')
print(f'r2: {model.summary.r2}')
print(f'iterations: {model.summary.totalIterations}')

RMSE: 200735.27852107986
r2: 0.7010253344728111
iterations: 37


In [37]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator

In [38]:
df_train,df_test = df2.randomSplit([0.7,0.3],seed=1)


In [39]:
input_cols = df_train.columns
input_cols.remove("price")

va = VectorAssembler(
    inputCols=input_cols,
    outputCol="features"
)

lr = LinearRegression()\
  .setMaxIter(100)\
  .setRegParam(3.0)\
  .setElasticNetParam(0.5)\
  .setFeaturesCol("features")\
  .setLabelCol("price")




In [40]:
pipeline = Pipeline(stages=[va,lr])


In [41]:
model = pipeline.fit(df_train)
 
df_train_pred = model.transform(df_train)
df_test_pred = model.transform(df_test)

In [42]:
evaluator = RegressionEvaluator() \
                .setLabelCol("price") \
                .setPredictionCol("prediction") \
                .setMetricName("rmse") \
                
            
rmse = evaluator.evaluate(df_train_pred)
rmse_test = evaluator.evaluate(df_test_pred)


In [43]:
evaluator_r2 = RegressionEvaluator() \
                .setLabelCol("price") \
                .setPredictionCol("prediction") \
                .setMetricName("r2")

r2 = evaluator_r2.evaluate(df_train_pred)
r2_test = evaluator_r2.evaluate(df_test_pred)

In [44]:
print("RMSE:", rmse)
print("R2:", r2)
print("-------------test--------------")
print("RMSE:", rmse_test)
print("R2:", r2_test)

RMSE: 204884.06353335246
R2: 0.6979612248668865
-------------test--------------
RMSE: 191057.3640296164
R2: 0.7077400039432729


In [45]:
def pipeline_f(df):
    input_cols = df_train.columns
    input_cols.remove("price")

    va = VectorAssembler(
    inputCols=input_cols,
    outputCol="features"
)

    lr = LinearRegression()\
        .setMaxIter(100)\
        .setRegParam(3.0)\
        .setElasticNetParam(0.5)\
        .setFeaturesCol("features")\
        .setLabelCol("price")

    pipeline = Pipeline(stages=[va,lr])
    return pipeline

In [46]:
def evaluation_f(train,test):
    evaluator = RegressionEvaluator() \
                .setLabelCol("price") \
                .setPredictionCol("prediction") \
                .setMetricName("rmse") \
    
    evaluator_r2 = RegressionEvaluator() \
                .setLabelCol("price") \
                .setPredictionCol("prediction") \
                .setMetricName("r2")
            
    rmse = evaluator.evaluate(train)
    rmse_test = evaluator.evaluate(test)
    
    r2 = evaluator_r2.evaluate(train)
    r2_test = evaluator_r2.evaluate(test)
    results = [rmse, rmse_test, r2, r2_test]
    return results


In [47]:
pipeline = pipeline_f(df2)

In [48]:
k1 = [x for x in range(1,10)]
k2 = [x for x in range(10,100,10)]
k3 = [x for x in range(100,400,100)]

k = k1+k2+k3
result_columns = ['k', 'Test Size', 'RMSE Train', 'RMSE Test', 'R2 Train', 'R2 Test']
results = pd.DataFrame(columns=result_columns)


for k_val in k:
    test_size = 1 - k_val * len(df2.columns) / df2.count()
    df_train, df_test = df2.randomSplit([1 - test_size, test_size], seed=1)
    model = pipeline.fit(df_train)
    df_train_pred = model.transform(df_train)
    df_test_pred = model.transform(df_test)
    metrics = evaluation_f(df_train_pred, df_test_pred)
    results = results._append({'k': k_val,
                                    'Test Size': test_size,
                                    'RMSE Train': metrics[0],
                                    'RMSE Test': metrics[1],
                                    'R2 Train': metrics[2],
                                    'R2 Test': metrics[3]}, ignore_index=True)
    


/tmp/ipykernel_26/319405658.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = results._append({'k': k_val,
24/05/10 15:55:13 ERROR OWLQN: Failure! Resetting history: breeze.optimize.NaNHistory: 


In [49]:
results.head(100)

,k,Test Size,RMSE Train,RMSE Test,R2 Train,R2 Test
0,1.0,0.999075,42828.085051,437717.638044,0.982708,-0.421299
1,2.0,0.998149,94877.499789,394351.640120,0.874066,-0.152909
2,3.0,0.997224,102450.507959,266924.900091,0.827592,0.472105
3,4.0,0.996299,128759.362487,219465.187009,0.802963,0.643117
4,5.0,0.995373,134808.812971,233124.685112,0.805428,0.597306
5,6.0,0.994448,156682.999324,226705.711670,0.784919,0.618959
6,7.0,0.993522,162018.213214,222150.739827,0.755018,0.634263
7,8.0,0.992597,164698.336310,214471.369199,0.728894,0.659291
8,9.0,0.991672,158189.157618,215503.897993,0.728779,0.656234
9,10.0,0.990746,155554.118835,216798.816599,0.711116,0.652427


In [84]:
def evaluate_metrics(df,metrics=['r2','rmse','mse','mae'],label_col='price',prediction_col='prediction'):
    """
    Funkcja oblicza metryki regresji na podstawie zbioru danych zawierającego prawdziwe wartosci etykiet 
    i wartości przewidywane
    :param df: Dataset<Row> wejściowy zbiór danych
    :param metrics: lista metryk do obliczenia
    :param label_col: nazwa kolumny z wartosciami ściowymi
    :param prediction_col: nazwa kolumny zawierającej przewidywane wartosci
    :return: słownik zawierajacy pary (nazwa_metryki,wartosc) 
    """
    
    results = []
    evaluator = RegressionEvaluator() \
        .setLabelCol(label_col) \
        .setPredictionCol(prediction_col)
        
    
    for metric in metrics:
        result = evaluator.setMetricName(metric).evaluate(df)
        results.append(result)

    return results

In [125]:
def train_and_test(df,lr=LinearRegression()
                 ř         .setMaxIter(100)
                          .setRegParam(1.0)
                          .setElasticNetParam(0.5)
                          .setFeaturesCol("features")
                          .setLabelCol("price")):
    """
    Funkcja (1) dzieli zbiór danych na ``df_train`` oraz ``df_test`` 
    (2) tworzy ciag przetwarzania zawierający ``VectorAssembler`` 
    i  przekazaną jako parametr konfiguracje algorytmu regresji
    (3) buduje model i dokonuje predykcji dla ``df_train`` oraz ``df_test``
    (4) wyświetla metryki
    :param df: wejściowy zbiór danych
    :param lr: konfiguracja algorytmu regresji
    :return: model  zwrócony przez ``pipeline.fit()``
    :return: results  zwraca dataframe z wynikami metryki``
    """

    df_train,df_test = df.randomSplit([0.75,0.25],seed=1)
    input_cols = df_train.columns
    input_cols.remove("price")

    va = VectorAssembler(
        inputCols=input_cols,
        outputCol="features")
    
    pipeline = Pipeline(stages=[va,lr])
    model = pipeline.fit(df_train)

    df_train_pred = model.transform(df_train)
    df_test_pred = model.transform(df_test)

    result_columns = ['r2 Train', 'r2 Test', 'rmse Train', 'rmse Test']
    results = pd.DataFrame(columns=result_columns)
    
    metrics = evaluate_metrics(df_train_pred)
    metrics2 = evaluate_metrics(df_test_pred)
    results = results._append({'r2 Train': metrics[0],
                                    'r2 Test': metrics2[0],
                                    'rmse Train': metrics[1],
                                    'rmse Test': metrics2[1], 
                                    'mse Train': metrics[2],
                                    'mse Test': metrics2[2],
                                    'mae Train': metrics[3],
                                    'mae Test': metrics2[3]},ignore_index=True)

    
    return model,results


In [128]:
model,results = train_and_test(df2)
results.head()

/tmp/ipykernel_26/590277158.py:38: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = results._append({'r2 Train': metrics[0],


,r2 Train,r2 Test,rmse Train,rmse Test,mse Train,mse Test,mae Train,mae Test
0,0.697395,0.711583,203921.553092,191285.809619,4.158400e+10,3.659026e+10,126331.645008,126126.63388


In [129]:
coefficients = model.stages[1].coefficients
intercept = model.stages[1].intercept
features = df2.columns
features.remove("price")
num_features = len(coefficients)

print("price = \n")
for i, coef in enumerate(coefficients):
    print(f" + {features[i]} * {coef}\n")
    
print(f" + {intercept}")



price = 

 + bedrooms * -37602.92107672266

 + bathrooms * 44368.26926906392

 + sqft_living * 97.26278631159406

 + sqft_lot * 0.12770931755794784

 + floors * 4422.0130784428675

 + waterfront * 614587.3762614889

 + view * 51239.46072018579

 + condition * 26902.50711085491

 + grade * 91008.45316644131

 + sqft_above * 92.77853364870352

 + sqft_basement * 50.94380740423701

 + yr_built * -2633.577591664244

 + yr_renovated * 19.892407280051327

 + zipcode * -622.8654998578461

 + lat * 612888.9366825244

 + long * -235006.97748321737

 + sqft_living15 * 22.576338444173516

 + sqft_lot15 * -0.4067695729413095

 + unixdate * 0.001240435516700619

 + 5990365.362349896


In [9]:
df2 = df2.withColumn("date", from_unixtime(col("unixdate")).cast("date"))

print("bathrooms")
print(df2.agg({'bathrooms':'max'}).collect()[0][0])
print(df2.agg({'bathrooms':'median'}).collect()[0][0])
print("sqft_lot")
print(df2.agg({'sqft_lot':'mean'}).collect()[0][0])
print("condition")
print(df2.agg({'condition':'median'}).collect()[0][0])
print("bedrooms")
print(df2.agg({'bedrooms':'max'}).collect()[0][0])
print(df2.agg({'bedrooms':'median'}).collect()[0][0])
print(df2.agg({'bedrooms':'mean'}).collect()[0][0])
print(df2.agg({'bedrooms':'min'}).collect()[0][0])


bathrooms
8.0
2.25
sqft_lot
15106.967565816869
condition
3.0
bedrooms
33
3.0
3.37084162309721
0


In [10]:
df_filtered = df2.filter((col('bathrooms') >= 2) & (col('bathrooms') <= 3))
df_filtered = df_filtered.filter((col('sqft_lot') >= 15106 - 5000) & (col('sqft_lot') <= 15106 + 5000))
df_filtered = df_filtered.filter((col('condition') == 3.0))
df_filtered = df_filtered.filter((col('bedrooms') >= 2) & (col('bedrooms') <= 4))


In [11]:
pdf = df_filtered.toPandas()
pdf.head(100)

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,unixdate,date
0,395000.0,3,2.00,1890,14040,2.0,0,0,3,7,...,0,1994,0,98019,47.7277,-121.962,1890,14018,1406764800,2014-07-31
1,488000.0,3,2.50,3160,13603,2.0,0,0,3,8,...,0,2003,0,98019,47.7443,-121.977,3050,9232,1405641600,2014-07-18
2,975000.0,4,2.50,2720,11049,2.0,0,0,3,10,...,0,1989,0,98004,47.5815,-122.192,2750,11049,1410220800,2014-09-09
3,905000.0,4,2.50,3300,10250,1.0,0,0,3,7,...,910,1946,1991,98040,47.5873,-122.249,1950,6045,1410134400,2014-09-08
4,480000.0,4,2.50,3230,16171,2.0,0,3,3,9,...,710,2001,0,98001,47.3183,-122.253,2640,8517,1414454400,2014-10-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,210000.0,3,2.00,1440,10111,1.0,0,0,3,7,...,0,1963,0,98001,47.3450,-122.283,1580,10200,1417478400,2014-12-02
96,550000.0,3,2.25,2160,15360,1.0,0,0,3,8,...,750,1965,2000,98007,47.6232,-122.138,2180,8480,1410998400,2014-09-18
97,780000.0,4,2.25,2260,16188,1.0,0,0,3,10,...,0,1984,0,98006,47.5637,-122.112,2840,10158,1427328000,2015-03-26
98,402500.0,4,2.50,2600,11951,2.0,0,0,3,10,...,0,1988,0,98023,47.3053,-122.340,2820,12093,1399593600,2014-05-09


In [12]:
df_filtered = df_filtered.withColumn("price_per_sqft", col("price") / col("sqft_lot"))


In [13]:
df_no_waterfront = df_filtered.filter(col("waterfront") == 0)
df_waterfront = df_filtered.filter(col("waterfront") == 1)

df_no_waterfront = df_no_waterfront.withColumn("price_per_sqft_no_waterfront", col("price") / col("sqft_lot"))
df_waterfront = df_waterfront.withColumn("price_per_sqft_waterfront", col("price") / col("sqft_lot"))
